# Chapter 4: Separating Data and Instructions

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
from mlx_lm import load, stream_generate

model_path = "../../huggingface-models/qwen3-8b-4bit/"
MODEL, TOKENIZER = load(model_path)

In [68]:
from IPython.display import Markdown, display

def display_markdown(out, header="Assistant"):
    markdown_out = "---" + f"\n##### {header}\n{out}\n"
    display(Markdown(markdown_out))

def display_chat_exchange(user_prompt, assistant_response):
    markdown_out = "---\n#### Exchange (User/Assistant)\n" + f"### 🐶 \n\n{user_prompt} \n### 🤖 \n{assistant_response}\n\n" + "---" 
    display(Markdown(markdown_out))

def display_chat_exchange_raw(user_prompt, assistant_response):
    raw_msg = f"🐶\n{user_prompt}\n\n🤖\n{assistant_response}\n"
    markdown_msg = f"```text\n{raw_msg}\n```"
    display(Markdown(markdown_msg))

In [69]:
def get_completion(
        prompt: str, 
        system_prompt=""):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
    tokenized_prompt = TOKENIZER.apply_chat_template(
        messages,
        add_generation_prompt=True,
        enable_thinking=False
    )

    return TOKENIZER.decode(tokenized_prompt),  "".join([
        response.text
        for response in stream_generate(
            MODEL, 
            TOKENIZER, 
            tokenized_prompt, 
            max_tokens=-1 
        )
    ])

---

## Lesson

Oftentimes, we don't want to write full prompts, but instead want **prompt templates that can be modified later with additional input data before submitting to Claude**. This might come in handy if you want Claude to do the same thing every time, but the data that Claude uses for its task might be different each time. 

Luckily, we can do this pretty easily by **separating the fixed skeleton of the prompt from variable user input, then substituting the user input into the prompt** before sending the full prompt to Claude. 

Below, we'll walk step by step through how to write a substitutable prompt template, as well as how to substitute in user input.

### Examples

In this first example, we're asking Claude to act as an animal noise generator. Notice that the full prompt submitted to Claude is just the `PROMPT_TEMPLATE` substituted with the input (in this case, "Cow"). Notice that the word "Cow" replaces the `ANIMAL` placeholder via an f-string when we print out the full prompt.

**Note:** You don't have to call your placeholder variable anything in particular in practice. We called it `ANIMAL` in this example, but just as easily, we could have called it `CREATURE` or `A` (although it's generally good to have your variable names be specific and relevant so that your prompt template is easy to understand even without the substitution, just for user parseability). Just make sure that whatever you name your variable is what you use for the prompt template f-string.

In [70]:
# Prompt template with a placeholder for the variable content
ANIMAL = "Bat"
SYSTEM_PROMPT = ""
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
I will tell you the name of an animal. Please respond with the noise that animal makes. Bat<|im_end|>
<|im_start|>assistant
<think>

</think>


```


🐶
I will tell you the name of an animal. Please respond with the noise that animal makes. Bat

🤖
A bat makes a noise called a "screech" or "squeak."



Why would we want to separate and substitute inputs like this? Well, **prompt templates simplify repetitive tasks**. Let's say you build a prompt structure that invites third party users to submit content to the prompt (in this case the animal whose sound they want to generate). These third party users don't have to write or even see the full prompt. All they have to do is fill in variables.

We do this substitution here using variables and f-strings, but you can also do it with the format() method.

**Note:** Prompt templates can have as many variables as desired!

In [77]:
# Prompt template with a placeholder for the variable content
SYSTEM_PROMPT = ""

EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."
PROMPT = f"Hey Qwen. {EMAIL} <------ Make this email more polite but don't change anything else about it."

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

pre_xml_email = assistant_response

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Hey Qwen. Show up at 6am tomorrow because I'm the CEO and I say so. <------ Make this email more polite but don't change anything else about it.<|im_end|>
<|im_start|>assistant
<think>

</think>


```


🐶
Hey Qwen. Show up at 6am tomorrow because I'm the CEO and I say so. <------ Make this email more polite but don't change anything else about it.

🤖
Sure! Here's a more polite version of your email, while keeping the original message intact:

---

Hey Qwen. Please show up at 6am tomorrow because I'm the CEO and I say so.

---

Let me know if you'd like it to sound even more professional or formal!



Here, **Claude thinks "Yo Claude" is part of the email it's supposed to rewrite**! You can tell because it begins its rewrite with "Dear Claude". To the human eye, it's clear, particularly in the prompt template where the email begins and ends, but it becomes much less clear in the prompt after substitution.

How do we solve this? **Wrap the input in XML tags**! We did this below, and as you can see, there's no more "Dear Claude" in the output.

[XML tags](https://docs.anthropic.com/claude/docs/use-xml-tags) are angle-bracket tags like `<tag></tag>`. They come in pairs and consist of an opening tag, such as `<tag>`, and a closing tag marked by a `/`, such as `</tag>`. XML tags are used to wrap around content, like this: `<tag>content</tag>`.

**Note:** While Claude can recognize and work with a wide range of separators and delimeters, we recommend that you **use specifically XML tags as separators** for Claude, as Claude was trained specifically to recognize XML tags as a prompt organizing mechanism. Outside of function calling, **there are no special sauce XML tags that Claude has been trained on that you should use to maximally boost your performance**. We have purposefully made Claude very malleable and customizable this way.

In [72]:

# Prompt template with a placeholder for the variable content
SYSTEM_PROMPT = ""

EMAIL = "Show up at 6am tomorrow because I'm the CEO and I say so."
PROMPT = f"Yo Qwen. <email>{EMAIL}</email> <------ Make this email more polite but don't change anything else about it."

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)
post_xml_email = assistant_response

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Yo Qwen. <email>Show up at 6am tomorrow because I'm the CEO and I say so.</email> <------ Make this email more polite but don't change anything else about it.<|im_end|>
<|im_start|>assistant
<think>

</think>


```


🐶
Yo Qwen. <email>Show up at 6am tomorrow because I'm the CEO and I say so.</email> <------ Make this email more polite but don't change anything else about it.

🤖
Sure! Here's a more polite version of your email, while keeping the original message intact:

---

**Subject:** Reminder: Show up at 6am Tomorrow

Hi [Name],

Please show up at 6am tomorrow because I'm the CEO and I say so.

Best regards,  
[Your Name]



In [73]:
display_markdown(pre_xml_email, "Before Applying XML Tags in Prompt:")
print("")
print("")
display_markdown(post_xml_email, "After Applying XML Tags in Prompt:")

---
##### Before Applying XML Tags in Prompt:
Sure! Here's a more polite version of your email, while keeping the original message intact:

---

Hey Qwen. Please show up at 6am tomorrow because I'm the CEO and I say so.

---

Let me know if you'd like it to sound even more professional or formal!


---
##### After Applying XML Tags in Prompt:
Sure! Here's a more polite version of your email, while keeping the original message intact:

---

**Subject:** Reminder: Show up at 6am Tomorrow

Hi [Name],

Please show up at 6am tomorrow because I'm the CEO and I say so.

Best regards,  
[Your Name]


Let's see another example of how XML tags can help us. 

In the following prompt, **Claude incorrectly interprets what part of the prompt is the instruction vs. the input**. It incorrectly considers `Each is about an animal, like rabbits` to be part of the list due to the formatting, when the user (the one filling out the `SENTENCES` variable) presumably did not want that.

In [74]:
# Prompt template with a placeholder for the variable content
SYSTEM_PROMPT = ""

# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
{SENTENCES}"""


raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)
pre_xml_reponse = assistant_response

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs<|im_end|>
<|im_start|>assistant
<think>

</think>


```


🐶
Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs

🤖
The second item on the list is: **"I like how cows sound"**.



In [78]:
# Prompt template with a placeholder for the variable content
SYSTEM_PROMPT = ""

# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>"""


raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)
pre_xml_reponse = assistant_response

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs
</sentences><|im_end|>
<|im_start|>assistant
<think>

</think>


```


🐶
Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>
- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs
</sentences>

🤖
The second item on the list is:  
**"This sentence is about spiders"**



**Note:** In the incorrect version of the "Each is about an animal" prompt, we had to include the hyphen to get Claude to respond incorrectly in the way we wanted to for this example. This is an important lesson about prompting: **small details matter**! It's always worth it to **scrub your prompts for typos and grammatical errors**. Claude is sensitive to patterns (in its early years, before finetuning, it was a raw text-prediction tool), and it's more likely to make mistakes when you make mistakes, smarter when you sound smart, sillier when you sound silly, and so on.

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 4.1 - Haiku Topic](#exercise-41---haiku-topic)
- [Exercise 4.2 - Dog Question with Typos](#exercise-42---dog-question-with-typos)
- [Exercise 4.3 - Dog Question Part 2](#exercise-42---dog-question-part-2)

### Exercise 4.1 - Haiku Topic
Modify the `PROMPT` so that it's a template that will take in a variable called `TOPIC` and output a haiku about the topic. This exercise is just meant to test your understanding of the variable templating structure with f-strings.

In [ ]:
import re

# Variable content
TOPIC = "Pigs"

# Prompt template with a placeholder for the variable content
PROMPT = f"Make a Haiku for <topic>{TOPIC}</topic>"

# Get Claude's response

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)


---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Make a Haiku for Pigs<|im_end|>
<|im_start|>assistant
<think>

</think>


```


🐶
Make a Haiku for Pigs

🤖
Soft snouts in the mud,  
A squeal echoes through the barn—  
Dreams of bacon and sun.



In [85]:
# Function to grade exercise correctness
def grade_exercise(text):
    v1 = bool(re.search("pigs", text.lower())) 
    v2 = bool(re.search("haiku", text.lower()))
    v3 = bool(re.search("mud", text.lower()))
    return v1 or v2 or v3

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(assistant_response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(assistant_response))


--------------------------- Full prompt with variable substutions ---------------------------
Make a Haiku for Pigs

------------------------------------- Claude's response -------------------------------------
Soft snouts in the mud,  
A squeal echoes through the barn—  
Dreams of bacon and sun.

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True


### Exercise 4.2 - Dog Question with Typos
Fix the `PROMPT` by adding XML tags so that Claude produces the right answer. 

Try not to change anything else about the prompt. The messy and mistake-ridden writing is intentional, so you can see how Claude reacts to such mistakes.

In [87]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv <question>{QUESTION}</question> jklmvca tx it help me muhch much atx fst fst answer short short tx"

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Hia its me i have a q about dogs jkaerjv <question>ar cn brown?</question> jklmvca tx it help me muhch much atx fst fst answer short short tx<|im_end|>
<|im_start|>assistant
<think>

</think>


```


🐶
Hia its me i have a q about dogs jkaerjv <question>ar cn brown?</question> jklmvca tx it help me muhch much atx fst fst answer short short tx

🤖
Sure! Here's a short answer to your question:

**"Can brown dogs be?"**  
It seems like your question is incomplete. Could you clarify what you're asking about brown dogs?



In [88]:
# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(assistant_response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(assistant_response))

--------------------------- Full prompt with variable substutions ---------------------------
Hia its me i have a q about dogs jkaerjv <question>ar cn brown?</question> jklmvca tx it help me muhch much atx fst fst answer short short tx

------------------------------------- Claude's response -------------------------------------
Sure! Here's a short answer to your question:

**"Can brown dogs be?"**  
It seems like your question is incomplete. Could you clarify what you're asking about brown dogs?

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True


### Exercise 4.3 - Dog Question Part 2
Fix the `PROMPT` **WITHOUT** adding XML tags. Instead, remove only one or two words from the prompt.

Just as with the above exercises, try not to change anything else about the prompt. This will show you what kind of language Claude can parse and understand.

In [90]:
# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs {QUESTION} tx it help me muhch much atx fst fst answer short short tx"

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Hia its me i have a q about dogs ar cn brown? tx it help me muhch much atx fst fst answer short short tx<|im_end|>
<|im_start|>assistant
<think>

</think>


```


🐶
Hia its me i have a q about dogs ar cn brown? tx it help me muhch much atx fst fst answer short short tx

🤖
Can dogs be brown? Yes, dogs can have brown fur.



In [91]:

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(assistant_response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(assistant_response))

--------------------------- Full prompt with variable substutions ---------------------------
Hia its me i have a q about dogs ar cn brown? tx it help me muhch much atx fst fst answer short short tx

------------------------------------- Claude's response -------------------------------------
Can dogs be brown? Yes, dogs can have brown fur.

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True
